# Playground

In [ ]:
from paoc.helper import get_input, parse_multiline_string as pms
lines = get_input()

In [ ]:
ex_lines = pms("""
-L|F7
7S-7|
L|7||
-L-J|
L|-JF
""")

In [ ]:
import re

In [ ]:
C = {
    '|': '\u2502',
    '-': '\u2500',
    'L': '\u2514',
    'J': '\u2518',
    '7': '\u2510',
    'F': '\u250C',
    '.': '.',
    'S': 'S'
}

for k, v in C.items():
    print(k, v)

In [ ]:
# # used for day10vis.txt
# for l in lines:
#     parsed = l
#     for k, v in C.items():
#         parsed = parsed.replace(k, v)
#     print(parsed)

In [ ]:
# up down left right
U, D, L, R = (-1, 0), (+1, 0), (0, -1), (0, +1)
dir_str = {(-1, 0): 'U', (+1, 0): 'D', (0, -1): 'L', (0, +1): 'R'}

dirs = {
    '|': {U, D},
    '-': {L, R},
    'L': {U, R},
    'J': {U, L},
    '7': {D, L},
    'F': {D, R},
    '.': None,
    'S': {U, D, L, R}
}

class Pipe:
    def __init__(self, char: str, r: int, c: int, came_from: tuple[int, int] = None) -> None:
        self.char = char
        self.r = r
        self.c = c
        if came_from is not None:
            self.cf_r, self.cf_c = came_from
            self.next_dir = (self.dirs - {(-self.cf_r, -self.cf_c)}).pop()

    @property
    def dirs(self) -> set[tuple[int, int]] | None:
        return dirs[self.char]
    
    def __matmul__(self, other: "Pipe") -> bool:
        assert abs(self.r-other.r + self.c-other.c) == 1, f'Pipes {self} & {other} are not neighbors'
        # other is up
        if other.r == self.r - 1:
            return U in self.dirs and D in other.dirs
        # other is down
        if other.r == self.r + 1:
            return D in self.dirs and U in other.dirs
        # other is left
        if other.c == self.c - 1:
            return L in self.dirs and R in other.dirs
        # other is right
        if other.c == self.c + 1:
            return R in self.dirs and L in other.dirs

    def __str__(self) -> str:
        return f'[ ({self.r}, {self.c}) \033[40m{C[self.char]}\033[49m ]'


grid = lines
def find_pipeline(grid: list[str], v=True) -> list[Pipe]:
    # find starting position
    for r, row in enumerate(grid):
        c = row.find('S')
        if c != -1:
            break

    start = Pipe('S', r, c)
    if v: print(start)
    iters = 0
    for i, start_dir in enumerate(start.dirs):
        r, c = start.r+start_dir[0], start.c+start_dir[1]
        pipe = Pipe(grid[r][c], r, c, came_from=start_dir)
        connected = start @ pipe
        if v: print(f'\n>>> {start} @ {pipe} : {connected}')
        if not connected:
            if v: print('NOT CONNECTED')
            continue
        pipeline = [start, pipe]
        # walk until we find ourselves
        while pipe.char != 'S':
            r, c = pipe.r+pipe.next_dir[0], pipe.c+pipe.next_dir[1]
            next_pipe = Pipe(grid[r][c], r, c, came_from=pipe.next_dir)
            connected = pipe @ next_pipe
            # print(f'{pipe} @ {next_pipe}: {connected}')
            if not connected:
                if v: print(f'{pipe} @ {next_pipe}: {connected}')
                if v: print('NOT CONNECTED')
                break
            pipe = next_pipe
            pipeline.append(pipe)
    return pipeline

pipeline = find_pipeline(lines, v=False)

# for pipe in pipeline:
    # print(pipe)

len(pipeline)//2

In [ ]:
# used for day10vis2.txt

positions = {(p.r, p.c) for p in pipeline}
from paoc.constants import INPUTS
with open(INPUTS / 'day10vis3.txt', 'w') as outfile:

    for r in range(len(grid)):
        for c in range(len(grid[r])):
            print(C[grid[r][c]] if (r, c) in positions else '.', end='', file=outfile)
        print(file=outfile)

In [ ]:
ex_lines = pms("""
FF7FSF7F7F7F7F7F---7
L|LJ||||||||||||F--J
FL-7LJLJ||||||LJL-77
F--JF--7||LJLJ7F7FJ-
L---JF-JLJ.||-FJLJJ7
|F|F-JF---7F7-L7L|7|
|FFJF7L7F-JF7|JL---7
7-L-JL7||F7|L7F-7F7|
L.L7LFJ|||||FJL7||LJ
L7JLJL-JLJLJL--JLJ.L
""")
for l in ex_lines:
    parsed = l
    for k, v in C.items():
        parsed = parsed.replace(k, v)
    print(parsed)

In [ ]:
doc = lines

pipeline = find_pipeline(doc, v=False)
# add sign attribute to all pipes
for p in pipeline:
    if p.char == 'S': continue
    # print(f'{p} ({p.cf_c}, {p.cf_r}) -> {p.next_dir}')
    p.sign = p.next_dir[0] if p.next_dir[0] != 0 else p.cf_r


In [ ]:
M = [['.' for _ in range(len(doc[0]))] for _ in range(len(doc))]

for p in pipeline:
    if p.char == 'S':
        M[p.r][p.c] = 'S'
        continue
    char = 'm' if p.sign == -1 else 'p' if p.sign == 1 else C[p.char]
    M[p.r][p.c] = char

for i in range(len(M)):
    M[i] = ''.join(M[i])


In [ ]:
O = {'p': 'm', 'm': 'p'}  # opposite

area = 0
for r, row in enumerate(M):
    # print(f'>>> {r}')
    # find first sign
    match = re.search('p|m', row)
    if not match:
        continue
    i, sign = match.start(), match.group()

    out = False

    while i < len(row):
        # find next instance off opposite sign
        match = re.search(O[sign], row[i:])
        if not match:
            break  # we're out of the loop here
        # portion of the row we'll be checking
        fr, to = i+1, i+match.start()
        slc = row[fr:to]
        n_dots = slc.count('.')
        if not out:
            area += n_dots
        # print(i, sign, f'({fr}, {to}) [{slc}]', n_dots, out)
        i += to - fr + 1
        sign = O[sign]
        out = not out
                
area